# Teoria da Dualidade

## Primal

Max: $z = 4x_1 + x_2 + 5x_3 + 3x_4$

S.a: 

\begin{align}
x_1 - x_2 - x_3 + 3x_4 \le 1 \\
5x_1 + x_2 + 3x_3 + 8x_4 \le 55 \\
-x_1 + 2x_2 + 3x_3 - 5x_4 \le 3 \\
x_1, x_2, x_3, x_4 \ge 0
\end{align}

In [ ]:
#include<iostream>

In [ ]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

In [ ]:
{
  // 1. Escolha o solver.
  MPSolver solver("programa_simples", MPSolver::GLOP_LINEAR_PROGRAMMING);

  // 2. Crie as variáveis: 
  auto const x1 = solver.MakeNumVar(0.0, solver.infinity(), "x1");
  auto const x2 = solver.MakeNumVar(0.0, solver.infinity(), "x2");
  auto const x3 = solver.MakeNumVar(0.0, solver.infinity(), "x3");
  auto const x4 = solver.MakeNumVar(0.0, solver.infinity(), "x4");

  std::cout << "Número de variáveis = " << solver.NumVariables() << std::endl;

  // 3. Defina as restrições: 
  auto const ct1 = solver.MakeRowConstraint(-solver.infinity(), 1.0, "ct1");
  ct1->SetCoefficient(x1, 1.0);
  ct1->SetCoefficient(x2, -1.0);
  ct1->SetCoefficient(x3, -1.0);
  ct1->SetCoefficient(x4, 3.0);

  auto const ct2 = solver.MakeRowConstraint(-solver.infinity(), 55.0, "ct2");
  ct2->SetCoefficient(x1, 5.0);
  ct2->SetCoefficient(x2, 1.0);
  ct2->SetCoefficient(x3, 3.0);
  ct2->SetCoefficient(x4, 8.0);
    
  auto const ct3 = solver.MakeRowConstraint(-solver.infinity(), 3.0, "ct3");
  ct3->SetCoefficient(x1, -1.0);
  ct3->SetCoefficient(x2, 2.0);
  ct3->SetCoefficient(x3, 3.0);
  ct3->SetCoefficient(x4, -5.0);
  std::cout << "Número de restrições = " << solver.NumConstraints() << std::endl;

  // 4. Defina a função objetivo: 
  auto const objective = solver.MutableObjective();
  objective->SetCoefficient(x1, 4.0);
  objective->SetCoefficient(x2, 1.0);
  objective->SetCoefficient(x3, 5.0);
  objective->SetCoefficient(x4, 3.0);
  objective->SetMaximization();

  // 5. Execute o solver e exiba os resultados.
  auto status = solver.Solve();
  if (status == MPSolver::OPTIMAL)
  {
      std::cout << "Solução:" << std::endl;
      std::cout << "Valor da função objetivo = " << objective->Value() << std::endl;
      std::cout << "x1 = " << x1->solution_value() << std::endl;
      std::cout << "x2 = " << x2->solution_value() << std::endl;
      std::cout << "x3 = " << x3->solution_value() << std::endl;
      std::cout << "x4 = " << x4->solution_value() << std::endl;
  }
}

## Dual

Min: $w = y_1 + 55y_2 + 3y_3$

S.a: 

\begin{align}
 y_1 + 5y_2 -  y_3 \ge 4 \\
-y_1 +  y_2 + 2y_3 \ge 1 \\
-y_1 + 3y_2 + 3y_3 \ge 5 \\
3y_1 + 8y_2 - 5y_3 \ge 3 \\
y_1, y_2, y_3 \ge 0
\end{align}

In [ ]:
{
  // 1. Escolha o solver.
  MPSolver solver("programa_simples", MPSolver::GLOP_LINEAR_PROGRAMMING);

  // 2. Crie as variáveis: 
  auto const y1 = solver.MakeNumVar(0.0, solver.infinity(), "y1");
  auto const y2 = solver.MakeNumVar(0.0, solver.infinity(), "y2");
  auto const y3 = solver.MakeNumVar(0.0, solver.infinity(), "y3");
  
  std::cout << "Número de variáveis = " << solver.NumVariables() << std::endl;

  // 3. Defina as restrições: 
  auto const ct1 = solver.MakeRowConstraint(4.0, solver.infinity(), "ct1");
  ct1->SetCoefficient(y1, 1.0);
  ct1->SetCoefficient(y2, 5.0);
  ct1->SetCoefficient(y3, -1.0);

  auto const ct2 = solver.MakeRowConstraint(1.0, solver.infinity(), "ct2");
  ct2->SetCoefficient(y1, -1.0);
  ct2->SetCoefficient(y2, 1.0);
  ct2->SetCoefficient(y3, 2.0);
    
  auto const ct3 = solver.MakeRowConstraint(5.0, solver.infinity(), "ct3");
  ct3->SetCoefficient(y1, -1.0);
  ct3->SetCoefficient(y2, 3.0);
  ct3->SetCoefficient(y3, 3.0);

  auto const ct4 = solver.MakeRowConstraint(3.0, solver.infinity(), "ct4");
  ct4->SetCoefficient(y1, 3.0);
  ct4->SetCoefficient(y2, 8.0);
  ct4->SetCoefficient(y3, -5.0);

  std::cout << "Número de restrições = " << solver.NumConstraints() << std::endl;

  // 4. Defina a função objetivo: 
  auto const objective = solver.MutableObjective();
  objective->SetCoefficient(y1, 1.0);
  objective->SetCoefficient(y2, 55.0);
  objective->SetCoefficient(y3, 3.0);
  objective->SetMinimization();

  // 5. Execute o solver e exiba os resultados.
  auto status = solver.Solve();
  if (status == MPSolver::OPTIMAL)
  {
      std::cout << "Solução:" << std::endl;
      std::cout << "Valor da função objetivo = " << objective->Value() << std::endl;
      std::cout << "y1 = " << y1->solution_value() << std::endl;
      std::cout << "y2 = " << y2->solution_value() << std::endl;
      std::cout << "y3 = " << y3->solution_value() << std::endl;
  }
}